In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
 
folder = '/content/drive/MyDrive/assets/chess_dataset'
image_size = (120, 120)
batch_size = 32
channels = 3
 
import cv2
 
def blur(img):
    return (cv2.blur(img,(3, 3)))

 
from keras.preprocessing.image import ImageDataGenerator
 
datagen = ImageDataGenerator(
        preprocessing_function=blur,
        rotation_range=5,
        width_shift_range=0.025,
        height_shift_range=0.025,
        rescale=1./255,
        horizontal_flip=True,
        brightness_range=(0.8, 1.2),
        fill_mode='nearest')
 
test_datagen = ImageDataGenerator(
     preprocessing_function=blur,
     rescale=1./255)
 
train_gen = datagen.flow_from_directory(
    folder + '/train',
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    color_mode = 'rgb',
    shuffle=True,
    # save_to_dir=folder + '/generate'
)
 
test_gen = test_datagen.flow_from_directory(
    folder + '/test',
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    color_mode = 'rgb',
    shuffle=False,
)

# i = 0
# for batch in train_gen:
#     i += 1
#     if i > 5:
#         break


In [ ]:
train_gen.class_indices.keys()
test_gen.class_indices.keys()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.models import Model
from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.imagenet_utils import decode_predictions
 
b_model = ResNet50V2(include_top=False, weights='imagenet',
                     input_shape=(image_size[0],image_size[0],channels), pooling='max')
 
for layer in b_model.layers:
    layer.trainable = False    
 
x = b_model.output
x = Flatten()(x)   
x = Dense(500, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(500, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(160, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(80, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(13, activation='softmax')(x)
 
model = Model(inputs=b_model.input, outputs=x)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy', 'accuracy'])

In [ ]:
import os
import datetime
from keras.callbacks import ModelCheckpoint, TensorBoard
 
epochs = 11
 
dtb = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "/content/drive/MyDrive/outputs/logs/" + dtb
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
mc = ModelCheckpoint('/content/drive/MyDrive/outputs/models/best_model_{}.h5'.format(dtb), monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
 
history = model.fit(
                    train_gen,
                    validation_data=test_gen,
                    verbose = 1,
                    epochs=epochs,
                    callbacks=[tensorboard_callback, mc])
 
model.save_weights('/content/drive/MyDrive/outputs/models/model_VGG19_{}.h5'.format(dtb))
model.save(os.path.join('/content/drive/MyDrive/outputs/models', 'chess-piece-{}.model'.format(dtb)))

In [ ]:
import seaborn as sn
import matplotlib.pyplot as plt
import pandas as pd
 
plt.plot(history.history['categorical_accuracy'], '--')
plt.plot(history.history['val_categorical_accuracy'], '-')
 
plt.title('Categorical Accuracy vs Epoch')
plt.xlabel('Epoch')
plt.ylabel('Categorical Accuracy')
plt.legend(['Train', 'Validation']);

In [ ]:
plt.plot(history.history['loss'], '--')
plt.plot(history.history['val_loss'], '-')
 
plt.title('Loss vs Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation']);

In [ ]:
plt.plot(history.history['categorical_accuracy'], '--')
plt.plot(history.history['loss'], '--')
plt.plot(history.history['val_loss'], '-')
plt.plot(history.history['val_categorical_accuracy'], '-')
 
plt.title('Accuracy & Loss vs Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy & Loss')
plt.legend(['Train Accuracy',  'Train Loss', 'Validation Accuracy', 'Validation Loss']);

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
 
labels_names = ['bb', 'bk', 'bn', 'bp', 'bq', 'br', 'em', 'wb', 'wk', 'wn', 'wp', 'wq', 'wr']
 
test_gen.reset()
Y = model.predict_generator(test_gen)
cls = test_gen.cls[test_gen.index_array]
y = np.argmax(Y, axis= -1)
 
 
data = confusion_matrix(cls, y)
df_cm = pd.DataFrame(data, columns=labels_names, index = labels_names)
df_cm.index.name = 'Input'
df_cm.columns.name = 'Prediction'
plt.figure(figsize = (16,10))
sn.set(font_scale=1.0)
sn.heatmap(df_cm, cmap="Greens", annot=True,annot_kws={"size": 12})

In [ ]:
print(data)
print(classification_report(test_gen.classes[test_gen.index_array], y_pred, target_names=target_names))

In [ ]:
    print("Evaluate data")
    result = model.evaluate(test_gen, verbose=1)
    print("test loss, test acc:", result)